In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit

In [3]:
mpg = pd.read_csv("./mpg.csv", index_col="name") # load mpg dataset
mpg = mpg.loc[mpg["horsepower"] != '?'].astype(int) # remove columns with missing horsepower values
mpg_train, mpg_test = train_test_split(mpg, test_size = .2, random_state = 0) # split into training set and test set
mpg_train, mpg_validation = train_test_split(mpg_train, test_size = .5, random_state = 0)
mpg_train.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
name,,,,,,,,
toyota corolla,34,4,108,70,2245,16,82,3
buick century,17,6,231,110,3907,21,75,1
cadillac eldorado,23,8,350,125,3900,17,79,1
bmw 320i,21,4,121,110,2600,12,77,2
ford fairmont futura,24,4,140,92,2865,16,82,1


<a id='regularization'></a>
## Regularization

Last week we talked about validation as a means of combating overfitting. Another method is to add *regularization* terms to our loss function. **Regularization** basically penalizes complexity in our models. This allows us to add explanatory variables to our model without worrying as much about overfitting. Here's what our ordinary least squares model looks like with a regularization term:

$$\hat{\boldsymbol{\theta}} = \arg\!\min_\theta \sum_{i=0}^n (y_i - f_\boldsymbol{\theta}(x_i))^2 + \lambda R(\boldsymbol{\theta})$$

We've written the model a little differently here, but the first term is the same as the ordinary least squares regression model you learned last week. This time it's just generalized to any function of $x$ where $\theta$ is a list of parameters, or weights on our explanatory variables, such as coefficients to a polynomial. We're minimizing a loss function to find the best coefficients for our model. 

The second term is the **regularization** term. The $\lambda$ parameter in front of it dictates how much we care about our regularization term – the higher $\lambda$ is, the more we penalize large weights, and the more the regularization makes our weights deviate from OLS. 

**Question**: What happens when $\lambda = 0$?

So, what is $R(\theta)$? It could be a lot of things! Today we'll talk about two of the most common regularization functions – ridge and LASSO. 

<table>
    <tr><td>Ridge</td><td>L2 Norm</td><td>$R(\boldsymbol{\theta}) = \sum\limits_{i=0}^n \theta_i^2$</td></tr>
    <tr><td>LASSO</td><td>L1 Norm</td><td>$R(\boldsymbol{\theta}) = \sum\limits_{i=0}^n \lvert\theta_i\rvert$</td></tr>
</table>

<a id='ridge'></a>
### Ridge 
In **ridge** regression, the regularization function is the sum of squared weights. One of the nice things about ridge regression is that there is always a unique, mathematical solution that can be found using a known formula. The solution involves linear algebra, so you don't need to know it, but the existence of this formula also makes it computationally easy to solve.

$$\hat{\boldsymbol{\theta}} = \left(\boldsymbol{X}^T \boldsymbol{X} + \lambda\boldsymbol{I}\right)^{-1}\boldsymbol{X}^T\boldsymbol{Y}$$


In [ ]:
from sklearn.linear_model import Ridge

x_train = ...
y_train = ...

ridge_model = Ridge(alpha = 1.0)
ridge_model.fit(x_train, y_train)
loss = ... # mean squared error or ridge model
print("Root Mean Squared Error of ridge model: {:.2f}".format(loss))

<a id='lasso'></a>
### LASSO
In **LASSO** regression, the regularization function is the sum of absolute values of the weights. If there's one thing you should know about LASSO is that it is *sparsity inducing*. This just means that it forces some weights to take on zero values, leaving you with fewer explanatory variables in the resulting model than you put in. Unlike ridge regression, LASSO doesn't necessarily have a unique solution, and there's no formula that determines what the optimal weights should be.

In [ ]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha = 1.0)
lasso_model.fit(x_train, y_train)

loss = ...
print("Root Mean Squared Error of lasso model: {:.2f}".format(loss))

### Visualizing Ridge and LASSO
We just told you a lot of things about ridge and lasso, but here are some visualizations to help you understand the intuition behind some of the characteristics of these two regularization methods. Another way to describe the modified minimization function above is that it's the same loss function as before, with the *additional constraint* that $R(\boldsymbol{\theta}) \leq t$. Now, $t$ is related to $\lambda$ but the exact relationship between the two parameters depends on your data. Regardless, let's take a look at what this means in the two-dimensional case. For ridge,

$$\theta_0^2 + \theta_1^2 \leq t$$

Does this look familiar to you? What if it's in the form $x^2 + y^2 \leq t$? Or how about now:
<img src='http://vikingsseason5i.com/wp-content/uploads/2018/08/circle-equation-circle-equation-unit-circle.jpg' width=400 />

Lasso is of the form $$\left|\theta_0\right| + \left|\theta_1\right| \leq t$$ This one's a little harder to interpret, perhaps this will help inspire you:
<img src='https://cdn.kastatic.org/ka-perseus-graphie/3b9b8f4b4dac19e1197e9dd94553d0822f9fe69a.png' />

#### Norm Balls
<img src='https://upload.wikimedia.org/wikipedia/commons/f/f8/L1_and_L2_balls.svg' width=400/>
<img src='norm_balls.png' width=400/>

The rhombus and circle as a visualization of the regularization term, while the blue circles are the topological curves representing the loss function based on the weights. You want to minimize the sum of these, which means you want to minimize each of those. The point that minimizes the sum is the minimum point at which they intersect.


**Question**: Based on these visualizations, could you explain why LASSO is sparsity-inducing?

Turns out that the $L2-norm$ is always some sort of smooth surface, from a circle in 2D to a sphere in 3D. On the other hand, LASSO always has sharp corners. This is exactly the feature that makes it sparsiy-inducing. As you might imagine, just as humans are more likely to bump into sharp corners than smooth surfaces, the loss term is also most likely to intersect the $L2-norm$ at one of the corners.

### Regularization and Bias Variance
As we mentioned earlier, the bias is the average OLS loss term across multiple models of the same family (e.g. same degree polynomial) trained on separate datasets. Variance is the average variance of the weight vectors (coefficients) on your features. 

Without the regularization term, we’re just minimizing bias; the regularization term means we won’t get the lowest possible bias, but we’re exchanging that for some lower variance so that our model does better at generalizing to data points outside of our training data.


## Sanity Check

1. What happens as $\lambda$ increases?
    1. bias increases, variance increases
    2. bias increases, variance decreases
    3. bias decreases, variance increases
    4. bias decreases, variance decreases


2. **True** or **False**? Bias is how much error your model makes.


3. What is **sparsity**?


4. For each of the following, choose **ridge**, **lasso**, **both**, or **neither**:
    1. L1-norm
    2. L2-norm
    3. Induces sparsity
    4. Has analytic (mathematical) solution
    5. Increases bias
    6. Increases variance
    7. Results in lowest possible loss